In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import RobustScaler,MinMaxScaler,StandardScaler
from sklearn import preprocessing
from sklearn.model_selection import StratifiedKFold,RepeatedStratifiedKFold, RandomizedSearchCV,GridSearchCV, RepeatedKFold
from sklearn.metrics import (
    accuracy_score,
    f1_score,
    classification_report,
    confusion_matrix,
    roc_auc_score,
    roc_curve
)
from scikitplot.metrics import plot_precision_recall, plot_roc, plot_cumulative_gain, plot_lift_curve

In [ ]:
#Read datasets
df_train=pd.read_csv("../Dataset_prepared/Prepared_train.csv")
df_test=pd.read_csv("../Dataset_prepared/Prepared_test.csv")

#Drop useless features
df_train.drop('actor',axis=1,inplace=True)
df_train.drop('filename',axis=1,inplace=True)
df_test.drop('actor',axis=1,inplace=True)
df_test.drop('filename',axis=1,inplace=True)

#LabelEncode categorical features
le=preprocessing.LabelEncoder()
c = df_train.select_dtypes(include=['object']).columns.tolist()
for i in c:
    df_train[i]=le.fit_transform(df_train[i])

for i in c:
    df_test[i]=le.fit_transform(df_test[i])

#Normalize pure numeric features 
scaler = StandardScaler()
numeric_features = [n for n in df_train.columns if n!="emotion" and n!= "vocal_channel" and n!= "emotional_intensity" and n!= "statement" and n!= "repetition" and n!= "sex" and n!="filename" and n!="actor"]
scaled_features = scaler.fit_transform(df_train[numeric_features])
df_train[numeric_features] = scaled_features
scaled_features_test = scaler.transform(df_test[numeric_features])
df_test[numeric_features] = scaled_features_test

#Set X_train,y_train = data to fit models. Set X_test,y_test = data to test models.
col=[x for x in df_train.columns if x!="emotion"]
X_train=df_train[col].values
y_train = np.array(df_train["emotion"])
X_test=df_test[col].values
y_test = np.array(df_test["emotion"])

# Keras Deep Neural Networks

In [ ]:
from tensorflow import keras
from keras.models import Sequential # sequential networks= all the layers are one after the others
from keras.layers import Dense # we have flat layes 
from scikeras.wrappers import KerasClassifier

from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.regularizers import l2, l1
from keras.layers import Dropout
from keras.models import load_model

In [ ]:
# https://machinelearningmastery.com/tutorial-first-neural-network-python-keras/
# I modelli in keras sono definiti come una sequenza di livelli. 
# 1) Quindi creiamo un "Sequential model" e aggiungiamo i livelli uno alla volta
# 2) Nel primo livello dobbiamo inserire il numero di feature. Quindi assegnamo all'argomento "input_dim" 
# il numero di features da utilizzare
# Come configurare il numero di livelli e nodi in una NN
# https://machinelearningmastery.com/how-to-configure-the-number-of-layers-and-nodes-in-a-neural-network/
# 3) Utilizzamo una rete fully-connected, usando la classe "Dense"
# 4) Possiamo specificare il numero di neuroni o nodi nel livello con il primo argomento e specificare l'activation function
# La sigmoid sull'utimo livello ci assicura che l'output sia compreso tra 0 e 1
# 5) Durante la compilazione è necessario specificare alcune proprietà aggiuntive richieste per l'addestramento della rete
# Specifichiamo la funzione di perdita (loss) da utilizzare per valutare l'insieme di pesi
# "binary_crossentropy" è una funzione loss per classificazione binaria
# Come scegliere le loss functions: https://machinelearningmastery.com/how-to-choose-loss-functions-when-training-deep-learning-neural-networks/
# 6) Il training del modello si svolge su epoche e ogni epoca è suddivisa in lotti (batch):
# - Epoche: passa attraverso tutte le righe del training set
# - Batch: uno o più campioni considerati dal modello in un'epoca priam dell'aggiornamento dei pesi
# Differenza tra Epochs e Batch: https://machinelearningmastery.com/difference-between-a-batch-and-an-epoch/

Three different DNN models to fit with different batch size but same epochs

In [ ]:
n_classes = 8
def build_model1():
    n_feature = X_train.shape[1] # numero colonne
    model = Sequential()
    model.add(Dense(128, input_dim=n_feature, activation='relu')) 
    model.add(Dense(64, activation='relu'))# second layer
    model.add(Dense(n_classes, activation='softmax'))
    # if we have multiclass u have to specify as output the number of classes
    model.compile(loss='sparse_categorical_crossentropy', # compile=build the network. if binary classification use binary_crossentropy as loss fucntions
                  optimizer="adam", metrics=['f1_macro']) # metric u want to observe -> typically accuracy,f1 score
    return model

In [ ]:
n_classes = 8
def build_model2():
    n_feature = X_train.shape[1] # numero colonne
    model = Sequential()
    model.add(Dense(128, input_dim=n_feature, activation='tanh')) 
    model.add(Dense(64, activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(n_classes, activation='softmax'))
    # if we have multiclass u have to specify as output the number of classes
    model.compile(loss='sparse_categorical_crossentropy', # compile=build the network. if binary classification use binary_crossentropy as loss fucntions
                  optimizer="adam", metrics=['f1_macro']) # metric u want to observe -> typically accuracy,f1 score
    return model

In [ ]:
n_classes = 8
def build_model3():
    n_feature = X_train.shape[1] # numero colonne
    model = Sequential()
    model.add(Dense(64, input_dim=n_feature, activation='tanh')) 
    model.add(Dense(32, activation='relu'))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(n_classes, activation='softmax'))
    # if we have multiclass u have to specify as output the number of classes
    model.compile(loss='sparse_categorical_crossentropy', # compile=build the network. if binary classification use binary_crossentropy as loss fucntions
                  optimizer="adam", metrics=['f1_macro']) # metric u want to observe -> typically accuracy,f1 score
    return model

Now build the models with epochs=30

Now build the models with epochs=50

Now build the models with epochs=100

Try to build the best models with a grid search

In [ ]:
def build_model_best(meta, hidden_layer_sizes, activation):
    n_features_in_ = meta["n_features_in_"]
    n_classes_ = meta["n_classes_"]
    model = keras.models.Sequential()
    model.add(keras.layers.Input(shape=(n_features_in_,)))
    for hidden_layer_size in hidden_layer_sizes:
        model.add(keras.layers.Dense(hidden_layer_size, activation=activation))
    model.add(keras.layers.Dense(n_classes_, activation="softmax"))
    return model

In [ ]:
clf = KerasClassifier(
    model=build_model_best,
    loss="sparse_categorical_crossentropy",
    verbose=False
)

params = {
    'optimizer__learning_rate': [0.001, 0.01, 0.1, 1],
    'model__hidden_layer_sizes': [(128, 128, 128), (64, 64, 64), (32, 32, 32)],
    'model__activation': ['relu', 'tanh'],
    'optimizer': ["adam", "sgd"],
    'epochs': [10, 50, 100, 200]
}

best_model = RandomizedSearchCV(clf, params, scoring='f1_macro', n_jobs=-1, verbose=True, n_iter=30, cv=3)
best_model.fit(X_train, y_train)
print(best_model.best_score_, best_model.best_params_)

In [ ]:
y_pred = best_model.predict(X_test).astype(int)
print(classification_report(y_test, y_pred))